In [16]:
import sys
sys.path.insert(0, r'D:\Projects\aaa')

import pandas as pd
import matplotlib.pyplot as plt

import random
import numpy as np
random.seed(42)
np.random.seed() 

from src.utils.utils import get_files_in_from_directory

In [17]:
allx = r'D:\Projects\aaa\results\rq4_results\features.csv'
npm = r'D:\Projects\aaa\results\rq4_results\features_npm.csv'
pypi = r'D:\Projects\aaa\results\rq4_results\features_pypi.csv'
mvn = r'D:\Projects\aaa\results\rq4_results\features_mvn.csv'

df_all = pd.read_csv(pypi)

In [18]:
# Sanitize time_to_next_merge where nan is a valid value
max_merge = df_all['time_to_next_merge'].max() * 10
df_all.loc[df_all['time_to_next_merge'].isna(),'time_to_next_merge'] = max_merge

df_all.fillna(0, inplace=True)
df_all.fillna(0, axis=1, inplace=True)

In [19]:
print(df_all.shape[0])
print((df_all['label_security_related']==True).sum())

13175
268


In [20]:
_byrepo = df_all.groupby('label_repo_full_name')
repos = df_all['label_repo_full_name'].unique()
train_repos = random.sample(list(repos), int(0.9*len(repos)))
df = df_all[df_all.apply(lambda x: x['label_repo_full_name'] in train_repos, axis=1)]
eval_df = df_all[df_all.apply(lambda x: x['label_repo_full_name'] not in train_repos, axis=1)]

print(df.shape, (df['label_security_related']==True).sum())
print(eval_df.shape, (eval_df['label_security_related']==True).sum())


(12536, 161) 250
(639, 161) 18


In [21]:
# nan_ratio = {col:0 for col in df}

# for col in df:
#     nan_counts = df[col].isna().sum()
#     nan_ratio[col] = nan_counts/df.shape[0]

# nan_ratio = {k: v for k,v in sorted(nan_ratio.items(), key=lambda kv: -kv[1])}
# for k,v in nan_ratio.item():
#     if v > 0.1:
#         print(nan_ratio)


In [22]:
broken_features = [
    'commits_to_next_merge',
    'commits_since_last_merge',
    'commits_to_next_merge',
    'commits_since_last_merge',
    'file_changed_method_count_avg',
    'file_changed_method_count_max',
]

In [23]:
no_variance_features = [
    'deadlock_in_patch',
    'has_mvn_code',
    'has_mvn_like_code',
    'has_npm_like_code',
    'has_pypi_like_code',

    'deadlock_in_message',

    'methods_with_attack_count_avg',
    'methods_with_attack_count_max',
    'methods_with_corrupt_count_avg',
    'methods_with_crash_count_avg',
    'methods_with_deadlock_count_avg',
    'methods_with_exploit_count_avg',
    'methods_with_exploit_count_max',
    'methods_with_leak_count_avg',
    'methods_with_malicious_count_avg',
    'methods_with_malicious_count_max',
    'methods_with_segfault_count_avg',
    'methods_with_segfault_count_max',
    'malicious_in_patch',
    'segfault_in_patch',
]

In [24]:
highly_correlated_features = [
    'added_lines_ratio_avg',
    'added_lines_ratio_max',

    'avg_method_parameter_count_avg',
    'avg_method_parameter_count_max',
    'avg_method_token_count_avg',
    'avg_method_token_count_max',
    'avg_method_complexity_avg',

    'methods_with_vulnerab_count_max',
    'modified_lines_ratio_max',
    'max_method_parameter_count_max',

    'file_complexity_max',
    'file_complexity_avg',
    'max_method_nloc_avg',
    'max_method_nloc_max',
    'file_nloc_avg',
    'file_nloc_max',

    'secur_in_title',
    'vulnerab_in_title',
    'exploit_in_title',
    'certificat_in_title',
    'authent_in_title',
    'leak_in_title',
    'sensit_in_title',
    'crash_in_title',
    'attack_in_title',
    'deadlock_in_title',
    'segfault_in_title',
    'malicious_in_title',
    'corrupt_in_title',
    ]
    

In [25]:

df_selected = df
df_selected = df_selected[df_selected.columns.difference(broken_features)]
df_selected = df_selected[df_selected.columns.difference(no_variance_features)]
df_selected = df_selected[df_selected.columns.difference(highly_correlated_features)]

X = df_selected[df_selected.columns.difference(['label_repo_full_name', 'label_sha', 'label_commit_date', 'label_security_related'])]
y = df_selected['label_security_related']


In [26]:
from sklearn.feature_selection import VarianceThreshold
sel = VarianceThreshold(threshold=0.001)
sel.fit(X)
sup = sel.get_support()
i = -1
for x in X:
       i+=1
       if not sup[i]:
              print(f'reject \'{x}\',')


reject 'attack_in_title',
reject 'corrupt_in_message',
reject 'corrupt_in_title',
reject 'deadlock_in_title',
reject 'exploit_in_message',
reject 'exploit_in_title',
reject 'malicious_in_message',
reject 'malicious_in_title',
reject 'segfault_in_message',
reject 'segfault_in_title',


In [27]:
# columns = ['corrupt_in_message', 'deadlock_in_message', 'exploit_in_message', 'malicious_in_message', 'segfault_in_message']
# df_sec = df[df['label_security_related'] == True]
# for col in columns:
#     print(df_sec[col].sum())

In [28]:
correlation_matrix = X.corr()
corr = correlation_matrix.values
column_names = correlation_matrix.columns

for i in range(len(column_names)):
    for j in range(i+1, len(column_names)):
        if abs(corr[i,j])> 0.75:
            print('reject', column_names[i], ' ', column_names[j], ' ', corr[i,j])

reject added_lines_count_avg   added_lines_count_max   0.8946420745215757
reject added_lines_count_avg   changed_lines_count_max   0.8377765440316571
reject added_lines_count_max   changed_lines_count_max   0.9342811754748904
reject added_lines_ratio_avg   added_lines_ratio_max   0.7909161138486801
reject added_lines_ratio_avg   removed_lines_ratio_avg   -0.9898624144081328
reject added_lines_ratio_max   removed_lines_ratio_avg   -0.7789118279226971
reject authent_in_message   authent_in_title   0.8372416622156089
reject avg_method_complexity_avg   avg_method_nloc_avg   0.7653618291233982
reject avg_method_complexity_avg   max_method_complexity_avg   0.9017928232390353
reject avg_method_complexity_max   avg_method_nloc_max   0.8009976404570783
reject avg_method_nloc_avg   avg_method_token_count_avg   0.8585447726461289
reject avg_method_nloc_avg   max_method_nloc_avg   0.7607107750328206
reject avg_method_nloc_max   max_method_nloc_max   0.8144154857698844
reject avg_method_parameter_c

In [ ]:
print('Features selected')

In [ ]:
# import numpy as np

# # 2:4 or 1:2
# # manual rebalancing
# oversample_ratio = 2
# undersample_ratio = 4

# x_positive = X.where(y).dropna()
# x_negative = X.where(y==False).dropna()

# x_positive = pd.concat([x_positive for i in range(oversample_ratio)])
# x_negative = x_negative.sample(len(x_positive)*undersample_ratio)

# X_resampled = pd.concat([x_negative, x_positive])
# y_resampled = np.array([False]*len(x_negative) + [True]*len(x_positive))


In [ ]:
import numpy as np
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler, SMOTE, ADASYN


# 2:4 or 1:2
# rebalancing with SMOTE AID
oversample_ratio = 1
undersample_ratio = 2

sampler = SMOTE()

x_positive = X.where(y).dropna()
x_negative = X.where(y==False).dropna()

x_negative_temp = x_negative.sample(len(x_positive)*oversample_ratio)
X_resampled_temp = pd.concat([x_negative_temp, x_positive])
y_resampled_temp = np.array([False]*len(x_negative_temp) + [True]*len(x_positive))

X_resampled_temp, y_resampled_temp = sampler.fit_resample(X_resampled_temp, y_resampled_temp)

x_negative_temp2 = x_negative.sample(len(X_resampled_temp)*undersample_ratio)
X_resampled = np.concatenate([X_resampled_temp, x_negative_temp2])
y_resampled = np.concatenate([y_resampled_temp, [False]*(len(X_resampled_temp)*undersample_ratio)])


In [ ]:
print(x_positive.shape, x_negative.shape, X_resampled.shape)

In [ ]:
# scaling
from sklearn.preprocessing import RobustScaler

scaler = RobustScaler().fit(X_resampled)
X_resampled = scaler.transform(X_resampled)


In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression

from sklearn.feature_selection import RFECV
from sklearn.model_selection import KFold
from sklearn.model_selection import RandomizedSearchCV

from sklearn.metrics import recall_score, precision_score

VERBOSE_LVL = 10

In [20]:
# Train selector & RF
    
model = RandomForestClassifier(
    n_estimators = 100,
    random_state=42)

selector = RFECV(model, 
    step=1, 
    cv=5,
    min_features_to_select=40,
    scoring = 'f1',
    n_jobs=-1,
    verbose=VERBOSE_LVL)

selector.fit(X_resampled, y_resampled)

print('max f1:', max(selector.cv_results_['mean_test_score']))

Fitting estimator with 104 features.
Fitting estimator with 103 features.
Fitting estimator with 102 features.
Fitting estimator with 101 features.
Fitting estimator with 100 features.
Fitting estimator with 99 features.
Fitting estimator with 98 features.
Fitting estimator with 97 features.
Fitting estimator with 96 features.
Fitting estimator with 95 features.
Fitting estimator with 94 features.
Fitting estimator with 93 features.
Fitting estimator with 92 features.
Fitting estimator with 91 features.
Fitting estimator with 90 features.
Fitting estimator with 89 features.
Fitting estimator with 88 features.
Fitting estimator with 87 features.
Fitting estimator with 86 features.
Fitting estimator with 85 features.
Fitting estimator with 84 features.
Fitting estimator with 83 features.
Fitting estimator with 82 features.
Fitting estimator with 81 features.
Fitting estimator with 80 features.
Fitting estimator with 79 features.
Fitting estimator with 78 features.
Fitting estimator with 

In [21]:
#dict_keys(['mean_test_score', 'std_test_score', 'split0_test_score', 'split1_test_score', 'split2_test_score', 'split3_test_score', 'split4_test_score'])
# print(selector.cv_results_['mean_test_score'])

In [22]:
colums = list(X.columns)

print('Selected features')
for i in range(len(selector.support_)):
    if selector.support_[i]:
        print(colums[i])

Selected features
added_lines_count_avg
added_lines_count_max
attack_in_file_content
attack_in_message
authent_in_file_content
authent_in_message
authent_in_patch
author_in_top_100
author_to_commiter_date_diff
avg_method_complexity_max
avg_method_nloc_avg
avg_method_nloc_max
certificat_in_file_content
changed_files
changed_lines_count_avg
changed_lines_count_max
changed_methods_count_avg
changed_methods_count_max
changes_to_file_in_next_50_commits_avg
changes_to_file_in_next_50_commits_max
changes_to_file_in_prev_50_commits_avg
changes_to_file_in_prev_50_commits_max
commits_next_30_days
commits_next_7_days
commits_prev_7_days
corrupt_in_file_content
crash_in_file_content
dmm_unit_complexity
dmm_unit_interfacing
dmm_unit_size
file_size_avg
file_size_max
file_token_count_avg
file_token_count_max
has_pypi_code
is_add
is_file_recently_added
is_modify
leak_in_file_content
leak_in_message
malicious_in_file_content
max_method_complexity_avg
max_method_complexity_max
max_method_parameter_count

In [23]:
import numpy as np

df_eval_selected = eval_df
df_eval_selected = df_eval_selected[df_eval_selected.columns.difference(broken_features)]
df_eval_selected = df_eval_selected[df_eval_selected.columns.difference(no_variance_features)]
df_eval_selected = df_eval_selected[df_eval_selected.columns.difference(highly_correlated_features)]

eval_X = df_eval_selected[df_eval_selected.columns.difference(['label_repo_full_name', 'label_sha', 'label_commit_date', 'label_security_related'])]
eval_y = df_eval_selected['label_security_related']


eval_x_positive = eval_X.where(eval_y).dropna()
eval_x_negative = eval_X.where(eval_y==False).dropna()
print(eval_x_positive.shape, eval_x_negative.shape)

eval_x_negative_sampled =  eval_x_negative.sample(len(eval_x_positive)*undersample_ratio)

eval_X_balanced = pd.concat([eval_x_negative_sampled, eval_x_positive])
eval_y_balanced = np.array([False]*len(eval_x_negative_sampled) + [True]*len(eval_x_positive))


(32, 104) (1258, 104)


In [24]:
# eval RF selector

eval_scaled = scaler.transform(eval_X)
eval_selected_scaled = selector.transform(eval_scaled)
X_resampled_selected = selector.transform(X_resampled)
eval_X_balanced_selected = selector.transform(eval_X_balanced)

train_preds = selector.predict(X_resampled)
recall = recall_score(y_resampled, train_preds)
precision = precision_score(y_resampled, train_preds)
print('RF train', recall, precision)

eval_balanced_y_pred = selector.predict(eval_X_balanced)
recall = recall_score(eval_y_balanced, eval_balanced_y_pred)
precision = precision_score(eval_y_balanced, eval_balanced_y_pred)
print('RF test', recall, precision)

eval_y_pred = selector.predict(eval_scaled)
recall = recall_score(eval_y, eval_y_pred)
precision = precision_score(eval_y, eval_y_pred)

print('RF eval', recall, precision)


RF train 1.0 1.0
RF test 0.0 0.0
RF eval 0.15625 0.625


c:\Python310\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but RobustScaler was fitted without feature names
  warnings.warn(
c:\Python310\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but RFECV was fitted without feature names
  warnings.warn(
c:\Python310\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but RFECV was fitted without feature names
  warnings.warn(
c:\Python310\lib\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [25]:
# SVC_model = SVC(
#     kernel ='linear',
#     cache_size=1000,
#     random_state=42)

# SVC_model.fit(X_resampled_selected, y_resampled)


In [26]:
# SVC_model_2 = SVC(
#     kernel ='sigmoid',
#     random_state=42)

# SVC_model_2.fit(X_resampled_selected, y_resampled)

# train_preds = SVC_model_2.predict(X_resampled_selected)
# recall = recall_score(y_resampled, train_preds)
# precision = precision_score(y_resampled, train_preds)
# print('SVC2 train', recall, precision)

# eval_balanced_y_pred = SVC_model_2.predict(eval_X_balanced_selected)
# recall = recall_score(eval_y_balanced, eval_balanced_y_pred)
# precision = precision_score(eval_y_balanced, eval_balanced_y_pred)
# print('SVC2 test', recall, precision)

# eval_y_pred = SVC_model_2.predict(eval_selected_scaled)
# recall = recall_score(eval_y, eval_y_pred)
# precision = precision_score(eval_y, eval_y_pred)
# print('SVC2 eval', recall, precision)

In [27]:

# train_preds = SVC_model.predict(X_resampled_selected)
# recall = recall_score(y_resampled, train_preds)
# precision = precision_score(y_resampled, train_preds)
# print('SVC train', recall, precision)

# eval_balanced_y_pred = SVC_model.predict(eval_X_balanced_selected)
# recall = recall_score(eval_y_balanced, eval_balanced_y_pred)
# precision = precision_score(eval_y_balanced, eval_balanced_y_pred)
# print('SVC test', recall, precision)

# eval_y_pred = SVC_model.predict(eval_selected_scaled)
# recall = recall_score(eval_y, eval_y_pred)
# precision = precision_score(eval_y, eval_y_pred)
# print('SVC test', recall, precision)

In [28]:
LR_model = LogisticRegression(
    penalty ='l2',
    max_iter = 20000,
    random_state=42)

LR_model.fit(X_resampled_selected, y_resampled)


LogisticRegression(max_iter=20000, random_state=42)

In [29]:
train_preds = LR_model.predict(X_resampled_selected)
recall = recall_score(y_resampled, train_preds)
precision = precision_score(y_resampled, train_preds)
print('LR train', recall, precision)


eval_balanced_y_pred = LR_model.predict(eval_X_balanced_selected)
recall = recall_score(eval_y_balanced, eval_balanced_y_pred)
precision = precision_score(eval_y_balanced, eval_balanced_y_pred)
print('LR test', recall, precision)

eval_y_pred = LR_model.predict(eval_selected_scaled)
recall = recall_score(eval_y, eval_y_pred)
precision = precision_score(eval_y, eval_y_pred)
print('LR eval', recall, precision)

# eval_y_pred = LR_model.predict(eval_selected_scaled)
# recall = recall_score(eval_y, eval_y_pred)
# precision = precision_score(eval_y, eval_y_pred)
# print('LR', recall, precision)

LR train 0.18181818181818182 0.6919831223628692
LR test 0.4375 0.19444444444444445
LR eval 0.1875 0.4


In [30]:
print(len(eval_selected_scaled)) #3308
# (3308, 161) 45 => 

# 3076 commits ??

1290


In [31]:
eval_y_pred4 = selector.predict(eval_scaled)
eval_y_pred = [False]*len(eval_y_pred4)

for x in range(len(eval_y_pred4)):
    if eval_y_pred4[x]:
        eval_y_pred[x]=True

In [32]:
x_true = [False]*len(eval_y)
eval_y2 = eval_y.reset_index()
for x in range(len(eval_y2)):
    if eval_y_pred4[x] and eval_y2.iloc[x]['label_security_related']:
        x_true[x]=True
    
true_positives = sum(1  for x in x_true if x)
al_positives = (eval_df['label_security_related']==True).sum()
print(true_positives/al_positives) 


0.15625


In [33]:
import json

ix = 0
shas = []
for i, x in eval_df.iterrows():
    shas.append({
        'label_sha':x['label_sha'],
        'label_repo_full_name':x['label_repo_full_name'],
        'classification_pred': eval_y_pred[ix]
    })

    ix += 1

with open('classificated_commits.json', 'w') as f:
    json.dump(shas, f)


In [34]:
from joblib import dump, load

dump(LR_model, 'LR_model_all.joblib') 
# dump(SVC_model_2, 'SVC_model_2_all.joblib') 
# dump(SVC_model, 'SVC_model_all.joblib') 
dump(LR_model, 'LR_model_all.joblib') 



['LR_model_all.joblib']